In [341]:

import pandas as pd
import plotly.express as px
import plotly.io as pio


In [342]:
df = pd.read_csv("https://ourworldindata.org/grapher/annual-co2-emissions-per-country.csv?v=1&csvType=full&useColumnShortNames=true", storage_options = {'User-Agent': 'Our World In Data data fetch/1.0'})

In [343]:
print(df)

            Entity Code  Year  emissions_total
0      Afghanistan  AFG  1949          14656.0
1      Afghanistan  AFG  1950          84272.0
2      Afghanistan  AFG  1951          91600.0
3      Afghanistan  AFG  1952          91600.0
4      Afghanistan  AFG  1953         106256.0
...            ...  ...   ...              ...
29132     Zimbabwe  ZWE  2019       10262950.0
29133     Zimbabwe  ZWE  2020        8494503.0
29134     Zimbabwe  ZWE  2021       10203630.0
29135     Zimbabwe  ZWE  2022       10424940.0
29136     Zimbabwe  ZWE  2023       11164030.0

[29137 rows x 4 columns]


In [344]:
df = df[df['Year']>=1900]
print(df)

            Entity Code  Year  emissions_total
0      Afghanistan  AFG  1949          14656.0
1      Afghanistan  AFG  1950          84272.0
2      Afghanistan  AFG  1951          91600.0
3      Afghanistan  AFG  1952          91600.0
4      Afghanistan  AFG  1953         106256.0
...            ...  ...   ...              ...
29132     Zimbabwe  ZWE  2019       10262950.0
29133     Zimbabwe  ZWE  2020        8494503.0
29134     Zimbabwe  ZWE  2021       10203630.0
29135     Zimbabwe  ZWE  2022       10424940.0
29136     Zimbabwe  ZWE  2023       11164030.0

[23158 rows x 4 columns]


In [345]:
df = df.replace('',pd.NA)
df.dropna()
df = df[df['Code'].str.len() <= 3]


In [346]:
max_val = df["emissions_total"].max()
min_val = df["emissions_total"].min()
print(min_val, max_val)

0.0 11902503000.0


In [347]:
df = df.sort_values("Year")

In [348]:
ranges = [(0,5e6),(5e6, 5e7),(5e7,2.5e8),(2.5e8,1e9),(1e9,5e9),(5e9,1e10),(1e10,1.5e10),(1.5e10,2e10)]
colors = ["#ffffcc", "#ffeda0", "#fed976", "#feb24c", "#fd8d3c", "#fc4e2a", "#e31a1c", "#b10026"] 

discrete_labels = ["0-5M","5M-50M","50M-250M","250M-1B","1B-5B","5B-10B","10B-15B","15B+"]

In [349]:
def categorise_emissions(emissions_value):
    for i, (lower, upper) in enumerate(ranges):
        if lower <= emissions_value < upper:
            return discrete_labels[i]
    return[None]

df['emissions_bin'] = df['emissions_total'].apply(categorise_emissions)


In [350]:
df = df.dropna(subset=['emissions_bin'])
color_discrete_map = {label: color for label,color in zip(discrete_labels, colors)}


In [351]:
df = df.sort_values('Year')
df['Year']=pd.to_numeric(df['Year'])

In [352]:
pio.renderers.default = "browser"

In [353]:
dummy_data = []

for year in df['Year'].unique():
    for label in discrete_labels:
        dummy_data.append({
            'Entity': 'Dummy',
            'Year': year,
            'emission_total': 666,
            'Code': 'DUMMY',
            'emissions_bin': label,
        })

dummy_df = pd.DataFrame(dummy_data)

df = pd.concat([df,dummy_df])

In [354]:
fig = px.choropleth(
    df,
    color= "emissions_bin",
    locations = "Code",
    hover_name="Entity",
    hover_data={'emissions_total': True, "emissions_bin": False },
    animation_frame="Year",
    projection="natural earth",
    color_discrete_map=color_discrete_map

)


In [360]:
fig.update_layout(
    width=1000,  # Set width to a larger value
    height=600,  # Set height to a larger value
    updatemenus=[dict(
        type='buttons',
        showactive=False,
        buttons=[
            dict(
                label='Play',
                method='animate',
                args=[None, dict(frame=dict(duration=30, redraw=True), fromcurrent=True)]  # Set frame duration (in ms)
            ),
            dict(
                label='Pause',
                method='animate',
                args=[[None], dict(frame=dict(duration=0, redraw=False), mode='immediate', transition=dict(duration=0))]  # Pause animation
            )
        ]
    )],
    coloraxis_showscale=False,  # Hide default color scale
    legend_title="Emissions",  # Change the legend title manually
)


In [361]:
fig.write_html('Emissionsmap.html')